# Statistik-Projekt HS25 – Notebook 02: Deskriptive Statistik
## Ziel
In diesem Notebook berechnen wir zentrale statistische Kennzahlen für unsere Zielvariable `Ankunftsverspätung` (in Minuten).
Wir analysieren die Daten sowohl **global** (über alle Fahrten) als auch **gruppiert** nach unseren drei Kern-Hypothesen:

1.  **Tageszeit:** Unterscheidet sich die Verspätung je nach Tageszeit (Rush-Hour vs. Nacht)?
2.  **Linientyp:** Sind Fernverkehrszüge pünktlicher als S-Bahnen?
3.  **Wochentag:** Ist der Betrieb am Wochenende stabiler?

## Kennzahlen
Wir betrachten für jede Gruppe:
* **Mean (Mittelwert):** Durchschnittliche Verspätung.
* **Median (Zentralwert):** Robustes Maß für die "typische" Verspätung (weniger anfällig für Ausreißer).
* **Std (Standardabweichung):** Maß für die Streuung/Unzuverlässigkeit.
* **Pünktlichkeit:** Anteil der Züge mit < 3 Minuten Verspätung (SBB-Definition).
* **Ausreißer:** Min/Max Werte.

## Input
* Datei: `../data/processed/istdata_clean_extended.parquet` (Erstellt in Notebook 01)

In [2]:
import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path

# Konfiguration
INPUT_PATH = "../../data/processed/istdata_clean_extended.parquet"

# Prüfen
if not Path(INPUT_PATH).exists():
    raise FileNotFoundError(f"Datei nicht gefunden: {INPUT_PATH}. Bitte zuerst Notebook 01 ausführen!")

# Daten laden
print(f"Lade Daten aus {INPUT_PATH}...")
df = pl.read_parquet(INPUT_PATH)
print(f"Daten geladen: {len(df):,} Zeilen.")

# Schema prüfen (zur Sicherheit)
print("\nVerfügbare Spalten:", df.columns)

Lade Daten aus ../../data/processed/istdata_clean_extended.parquet...
Daten geladen: 4,201,596 Zeilen.

Verfügbare Spalten: ['BETRIEBSTAG', 'date_parsed', 'weekday_num', 'is_weekend', 'ANKUNFTSZEIT', 'hour_of_day', 'time_band', 'LINIEN_TEXT', 'linientyp', 'HALTESTELLEN_NAME', 'BPUIC', 'arr_delay_min', 'on_time', 'FAELLT_AUS_TF']


In [3]:
print("--- Globale Deskriptive Statistik (Alle Züge) ---")

# Wir nutzen Polars für schnelle Berechnung
global_stats = df.select([
    pl.col("arr_delay_min").mean().alias("Mean"),
    pl.col("arr_delay_min").median().alias("Median"),
    pl.col("arr_delay_min").std().alias("StdDev"),
    pl.col("arr_delay_min").min().alias("Min"),
    pl.col("arr_delay_min").max().alias("Max"),
    pl.col("arr_delay_min").quantile(0.25).alias("25%"),
    pl.col("arr_delay_min").quantile(0.75).alias("75%"),
    
    # Pünktlichkeit: Anteil Züge < 3 Min Verspätung (SBB Standard)
    ((pl.col("arr_delay_min") < 3).sum() / pl.len() * 100).alias("Pünktlichkeit (<3min) %")
])

# Schönere Darstellung mit Pandas
display(global_stats.to_pandas().round(2))

print("\nInterpretation:")
print(f"Im Durchschnitt sind Züge {global_stats['Mean'][0]:.2f} Minuten verspätet.")
print(f"Der Median liegt bei {global_stats['Median'][0]:.2f} Minuten. (Ist der Mean > Median? -> Rechtsschiefe Verteilung!)")

--- Globale Deskriptive Statistik (Alle Züge) ---


,Mean,Median,StdDev,Min,Max,25%,75%,Pünktlichkeit (<3min) %
0,0.93,0.62,2.11,-59.63,339.98,0.0,1.4,92.93



Interpretation:
Im Durchschnitt sind Züge 0.93 Minuten verspätet.
Der Median liegt bei 0.62 Minuten. (Ist der Mean > Median? -> Rechtsschiefe Verteilung!)


In [4]:
def analyze_group(df, group_col):
    """
    Berechnet statistische Kennzahlen gruppiert nach einer Spalte.
    """
    stats = (
        df.group_by(group_col)
        .agg([
            pl.len().alias("Anzahl_Fahrten"),
            pl.col("arr_delay_min").mean().alias("Mean"),
            pl.col("arr_delay_min").median().alias("Median"),
            pl.col("arr_delay_min").std().alias("StdDev"),
            
            # Robustes Streuungsmaß: IQR (Interquartilsabstand)
            (pl.col("arr_delay_min").quantile(0.75) - pl.col("arr_delay_min").quantile(0.25)).alias("IQR"),
            
            # Pünktlichkeit (< 3 min)
            ((pl.col("arr_delay_min") < 3).sum() / pl.len() * 100).alias("Pünktlichkeit_%")
        ])
        .sort(group_col)
    )
    
    # Rückgabe als Pandas DataFrame für schöne Formatierung
    return stats.to_pandas().round(2).set_index(group_col)

In [5]:
print("--- Analyse nach Tageszeit (Time Band) ---")

# Wir definieren eine logische Reihenfolge für die Sortierung (nicht alphabetisch)
time_order = ["Nacht", "Morgenpeak", "Tagesverkehr", "Abendpeak", "Spätabend"]

stats_time = analyze_group(df, "time_band")

# Sortieren nach unserer logischen Reihenfolge
stats_time = stats_time.reindex(time_order)

display(stats_time)

# Kurzer Check: Wo ist die Pünktlichkeit am schlechtesten?
worst_time = stats_time["Pünktlichkeit_%"].idxmin()
print(f"\nSchlechteste Pünktlichkeit: {worst_time} ({stats_time.loc[worst_time, 'Pünktlichkeit_%']}%)")

--- Analyse nach Tageszeit (Time Band) ---


,Anzahl_Fahrten,Mean,Median,StdDev,IQR,Pünktlichkeit_%
time_band,,,,,,
Nacht,270432,0.96,0.55,2.54,1.30,92.53
Morgenpeak,922309,0.99,0.70,2.01,1.45,92.67
Tagesverkehr,1349705,0.79,0.53,1.91,1.28,94.30
Abendpeak,948331,1.06,0.72,2.15,1.52,91.55
Spätabend,710819,0.93,0.57,2.36,1.38,92.67



Schlechteste Pünktlichkeit: Abendpeak (91.55%)


In [6]:
print("--- Analyse nach Linientyp (Fernverkehr vs. S-Bahn) ---")

stats_type = analyze_group(df, "linientyp")

# Sortieren nach Pünktlichkeit (aufsteigend)
stats_type = stats_type.sort_values("Pünktlichkeit_%")

display(stats_type)

# Differenz berechnen
if "Fernverkehr" in stats_type.index and "S-Bahn" in stats_type.index:
    diff = stats_type.loc["Fernverkehr", "Mean"] - stats_type.loc["S-Bahn", "Mean"]
    print(f"\nUnterschied im Mittelwert (Fernverkehr - S-Bahn): {diff:.2f} Minuten")

--- Analyse nach Linientyp (Fernverkehr vs. S-Bahn) ---


,Anzahl_Fahrten,Mean,Median,StdDev,IQR,Pünktlichkeit_%
linientyp,,,,,,
Andere,82284,1.45,0.13,5.21,1.43,86.61
Regional,1104395,0.94,0.57,2.32,1.67,90.89
Fernverkehr,294607,1.01,0.40,3.84,1.58,91.20
S-Bahn,2720310,0.90,0.65,1.51,1.27,94.14



Unterschied im Mittelwert (Fernverkehr - S-Bahn): 0.11 Minuten


In [7]:
print("--- Analyse nach Wochentag (Werktag vs. Wochenende) ---")

stats_weekend = analyze_group(df, "is_weekend")

# Index umbenennen für Lesbarkeit
stats_weekend.index = stats_weekend.index.map({True: "Wochenende", False: "Werktag"})

display(stats_weekend)

--- Analyse nach Wochentag (Werktag vs. Wochenende) ---


,Anzahl_Fahrten,Mean,Median,StdDev,IQR,Pünktlichkeit_%
is_weekend,,,,,,
Werktag,3147597,0.99,0.67,2.16,1.45,92.34
Wochenende,1053999,0.74,0.48,1.94,1.25,94.69


In [8]:
# Ordner für Reports erstellen
Path("../reports/tables").mkdir(parents=True, exist_ok=True)

stats_time.to_csv("../reports/tables/stats_tageszeit.csv")
stats_type.to_csv("../reports/tables/stats_linientyp.csv")
stats_weekend.to_csv("../reports/tables/stats_wochenende.csv")

print("Tabellen wurden im Ordner '../reports/tables/' gespeichert.")

Tabellen wurden im Ordner '../reports/tables/' gespeichert.
